In [1]:
from refine import *
import project
import mdtraj as mdt

from multiprocessing import Process, Queue, Lock
import multiprocessing

multiprocessing.set_start_method("fork")

from queue import Empty

%matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import style

style.use("fivethirtyeight")

Using matplotlib backend: TkAgg


In [2]:
project.setup()

In [35]:
def confined_gradient_descent(
        obj_q, decrement=0.9, termination="growth",
        absolute_bound=float("inf"), relative_bounds=(0.01, 7.0), etol=1, max_iter=100, return_traj=False):
    """
    Performs gradient descent of a system with respect to a special confinement.

    @param nmw: system to optimize.
    @type nmw: NMSpaceWrapper
    @param decrement: fold step when choosing optimal step.
    @type decrement: float
    @param termination: termination condition.
    @type termination: str
    @param absolute_bound: maximum rmsd between inital state and any intermediate state.
    @type absolute_bound: float
    @param relative_bounds: minimum and maximum rmsd between actual intermediate state and the next one.
    @type relative_bounds: tuple
    @param etol: terminates when |E(i+1) - E(i)| < etol
    @type etol: float
    @param max_iter: maximum number of iterations
    @type max_iter: int
    @param return_traj: if true all intermediate states, energies and forces are returned.
        Otherwise, the function returns only final record.
    @type return_traj: bool
    @return: dictionary containing all the results.
        "states" - list of all states along optimization path.
        "energies" - list of all energies along optimization path.
        "forces" - list of all forces along optimization path.
        If return_traj is false returns only last record.
    @rtype: dict
    """
    
    path = str((project.data_path / "may_complex") / "1dfj.pdb")
    print(path)
    create_system(path, tmp_file=str(project.output_path / "tmp_system.pdb"))
    t0 = time.time()
    drs = DRSystem(str(project.output_path / "tmp_system.pdb"), 'charmm36.xml', refine="chain A", static="chain B")
    print("construction of a system:", -t0 + time.time(), "sec")
    t0 = time.time()
    nmw1 = NMSpaceWrapper(drs, n_modes=10)
    print("INIT ENERGY:", nmw1.get_energy())
    
    m = 0
    
    while m != 10:
    
        natoms = len(nmw1.get_system_position())
        weights = np.ones((natoms,))
        
        # random noise
        preset_rmsd = .7

        # mu = a * mu_0
        # preset_rmsd = |a| (np.dot(mu_0, mu_0) / natoms) ** 0.5 => 
        # => a = (preset_rmsd ** 2 * natoms / np.dot(mu_0, mu_0)) ** 0.5

        mu = np.random.normal(0, 1, 10)
        mu = (preset_rmsd ** 2 * natoms / np.dot(mu, mu)) ** 0.5 * mu
        nmw1.set_position(np.zeros(10,))
        x1 = nmw1.get_system_position().copy()
        nmw1.set_position(mu)
        x2 = nmw1.get_system_position().copy()
        print("rmsd init:", rmsd(x1, x2, weights))
        print("RANDOM ENERGY:", nmw1.get_energy())
        pdy.writePDB(str(project.output_path / "shifted_6.0.pdb"), drs._refine_prot)

        nmw2 = NMSpaceWrapper(drs, n_modes=10)
        m = len(nmw2.get_eigenvalues())
    
    
    nmw2.set_position(np.zeros((10,)))
    nmw = nmw2
    
    # number of modes
    m = len(nmw.get_eigenvalues())
    # number of atoms
    n = len(nmw.get_system_position())
    # modes
    modes = nmw.get_modes()
    # special values
    ru_bound_value = n * relative_bounds[1] ** 2
    rl_bound_value = n * relative_bounds[0] ** 2
    a_bound_value = n * absolute_bound ** 2
    # initial state
    iteration_count = 0
    states = []
    energies = []
    forces = []
    print("hi e")
    energy_init = nmw.get_energy()
    print("hi p")
    position_init = nmw.get_position().copy()
    print("hi")
    energies.append(energy_init)
    states.append(nmw.get_system_position().copy())

    # it is a weighted anti-gradient!
    anti_gradient = nmw.get_force().copy()
    forces.append(anti_gradient)
    
    # update data
    print("hi")
    obj_q.put({
        "energies": energies,
        "states": states,
        "forces": forces,
    })
    
    # main cycle
    while True:
        print("\n(main) CYCLE START\n")

        # find a step
        # relative bound
        a = np.dot(anti_gradient, anti_gradient)
        upper_bound = (ru_bound_value / a) ** 0.5
        lower_bound = (rl_bound_value / a) ** 0.5
        # absolute bound


        # find optimal step
        dec = [1]
        eng = []
        ind = [0]

        nmw.set_position(dec[0] * upper_bound * anti_gradient + position_init)

        eng.append(nmw.get_energy())
        score = 0
        while True:
            dec.append(dec[-1] * decrement)
            ind.append(ind[-1] + 1)
            if dec[-1] * upper_bound <= lower_bound:
                break

            nmw.set_position(dec[-1] * upper_bound * anti_gradient + position_init)
            eng.append(nmw.get_energy())
            print("\t(line) energy:", eng[-1])
            # update data
            obj_q.put({
                "energies_line": eng,
            })
            
            # exit condition
            if energy_init > eng[-1] > eng[-2]:
                score += 1
            elif eng[-1] < eng[-2] or energy_init <= eng[-1]:
                score = 0
            if score >= 3:
                break

        # update state list
        j = np.argmin(eng)
        mu = dec[j] * upper_bound * anti_gradient

        # new initial states
        position_init = mu + position_init
        energy_init = eng[j]
        nmw.set_position(position_init)

        # state energy force
        states.append(nmw.get_system_position().copy())
        energies.append(eng[j])
        anti_gradient = nmw.get_force().copy()
        forces.append(anti_gradient)
        
        # print results
        print("(main) optimum:")
        print("       index:", j)
        print("       energy:", energies[-1])
        print("       force:", np.linalg.norm(forces[-1]))
        
        # adaptive upper bound
        ru_bound_value = min(np.dot(mu, mu) / decrement ** 3, n * relative_bounds[1] ** 2)
        print("       upper_bound:", (ru_bound_value / n) ** 0.5)
        
        # termination
        iteration_count += 1
        
        # update data
        obj_q.put({
            "energies": energies,
            "states": states,
            "forces": forces,
        })
        
        if termination == "growth":
            if energy_init < eng[j]:
                states.pop()
                energies.pop()
                forces.pop()
                break
        elif termination == "etol":
            if abs(eng[j] - energy_init) < etol:
                break
        else:
            raise ValueError(f"Wrong termination criterion: {termination}")
        if iteration_count >= max_iter:
            break
    if not return_traj:
        return {"states": states[-1:], "energies": energies[-1:]}
    else:
        return {"states": states, "energies": energies, "forces": forces}

In [4]:
path = str((project.data_path / "may_complex") / "1dfj.pdb")
print(path)
create_system(path, tmp_file=str(project.output_path / "tmp_system.pdb"))
t0 = time.time()
drs = DRSystem(str(project.output_path / "tmp_system.pdb"), 'charmm36.xml', refine="chain A", static="chain B")
print("construction of a system:", -t0 + time.time(), "sec")
t0 = time.time()
nmw1 = NMSpaceWrapper(drs, n_modes=10)
print("INIT ENERGY:", nmw1.get_energy())

@> 4416 atoms and 1 coordinate set(s) were parsed in 0.04s.


/home/semyon/PycharmProjects/DiplomaPython/data/may_complex/1dfj.pdb
write PDB(prody): 0.0438 sec
read PDB(openmm): 0.2772839069366455 sec


@> 8726 atoms and 1 coordinate set(s) were parsed in 0.06s.


add hydrogens and extra particles(openmm): 19.37400507926941 sec
write PDB(openmm): 0.0868675708770752 sec
1856 6870
construction of a system: 10.496461153030396 sec


@> Hessian was built in 8.08s.
@> 10 modes were calculated in 17.17s.


INIT ENERGY: -10889.754022221883


In [5]:
natoms = len(nmw1.get_system_position())
weights = np.ones((natoms,))

In [24]:
# random noise
preset_rmsd = .7

# mu = a * mu_0
# preset_rmsd = |a| (np.dot(mu_0, mu_0) / natoms) ** 0.5 => 
# => a = (preset_rmsd ** 2 * natoms / np.dot(mu_0, mu_0)) ** 0.5

mu = np.random.normal(0, 1, 10)
mu = (preset_rmsd ** 2 * natoms / np.dot(mu, mu)) ** 0.5 * mu
nmw1.set_position(np.zeros(10,))
x1 = nmw1.get_system_position().copy()
nmw1.set_position(mu)
x2 = nmw1.get_system_position().copy()
print("RMSD INIT:", rmsd(x1, x2, weights))
print("RANDOM ENERGY:", nmw1.get_energy())
print("NM VECTOR:", mu)
pdy.writePDB(str(project.output_path / "shifted_6.0.pdb"), drs._refine_prot)

rmsd init: 0.6999999999999991
RANDOM ENERGY: 58085336.94942912


'/home/semyon/PycharmProjects/DiplomaPython/output/shifted_6.0.pdb'

[ 10.20136524   8.53117505  12.69028443  -2.54902243  -1.86740009
 -13.92032945 -11.13093758  -8.52254014  -5.42638755 -11.90841998]


In [26]:
nmw2 = NMSpaceWrapper(drs, n_modes=10)

@> Hessian was built in 7.74s.
@> 10 modes were calculated in 14.41s.


In [27]:
old_coords = nmw2.get_system_position().copy()

In [29]:
nmw2.set_position(np.zeros((10,)))

58085336.94943367


In [36]:
# gradient descent
obj_q = Queue()
full_list = []
    
uprocess = Process(target=confined_gradient_descent, args=(obj_q,),
                   kwargs={"relative_bounds": (0.00001, 0.45),
                           "decrement": 0.9,
                           "max_iter": 100,
                           "return_traj": True,
                           "termination": "growth",
                          }, daemon=True)
uprocess.start()

anim = animation.FuncAnimation(fig, animate, 1000)

Using matplotlib backend: TkAgg
/home/semyon/PycharmProjects/DiplomaPython/data/may_complex/1dfj.pdb


@> 4416 atoms and 1 coordinate set(s) were parsed in 0.04s.


write PDB(prody): 0.0344 sec
read PDB(openmm): 0.2384171485900879 sec
add hydrogens and extra particles(openmm): 21.01369285583496 sec
write PDB(openmm): 0.11028242111206055 sec


@> 8726 atoms and 1 coordinate set(s) were parsed in 0.07s.


1856 6870
construction of a system: 10.317333936691284 sec


...

hi e
hi p
hi
hi

(main) CYCLE START



Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/lib/python3.6/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "/usr/lib/python3.6/tkinter/__init__.py", line 749, in callit
    func(*args)
  File "/home/semyon/mipt/GPCR-TEAM/env/lib/python3.6/site-packages/matplotlib/backends/_backend_tk.py", line 126, in _on_timer
    TimerBase._on_timer(self)
  File "/home/semyon/mipt/GPCR-TEAM/env/lib/python3.6/site-packages/matplotlib/backend_bases.py", line 1238, in _on_timer
    ret = func(*args, **kwargs)
  File "/home/semyon/mipt/GPCR-TEAM/env/lib/python3.6/site-packages/matplotlib/animation.py", line 1460, in _step
    still_going = Animation._step(self, *args)
  File "/home/semyon/mipt/GPCR-TEAM/env/lib/python3.6/site-packages/matplotlib/animation.py", line 1191, in _step
    self._draw_next_frame(framedata, self._blit)
  File "/home/semyon/mipt/GPCR-TEAM/env/lib/python3.6/site-packages/matplotlib/animation.py", line 1210, in _draw_

	(line) energy: 130271.46903765675
	(line) energy: 123895.1571812807
	(line) energy: 118609.91439443165
	(line) energy: 114216.01503675102
	(line) energy: 110552.31529611224
	(line) energy: 107488.44706715504
	(line) energy: 104918.63262100803
	(line) energy: 102756.84915154289
	(line) energy: 100933.03540199457
	(line) energy: 99390.12118245394
	(line) energy: 98081.84466985556
	(line) energy: 96971.34943390582
	(line) energy: 96030.68332353474
	(line) energy: 95241.63963058214
	(line) energy: 94598.56274643441
	(line) energy: 94114.03349940883
	(line) energy: 93828.63093398255
	(line) energy: 93825.67661730177
	(line) energy: 94251.37659119738
	(line) energy: 95339.18913553088
	(line) energy: 97435.10371178949
(main) optimum:
       index: 18
       energy: 93825.67661730177
       force: 62975.58631169573
       upper_bound: 0.07910681868513682

(main) CYCLE START

	(line) energy: 76587.695250136
	(line) energy: 78132.37344631185
	(line) energy: 79555.45249666169
(main) optimum:
   

       upper_bound: 0.00286320844851118

(main) CYCLE START

	(line) energy: 39751.50562096775
	(line) energy: 39751.46314724073
	(line) energy: 39751.501794768556
	(line) energy: 39751.59545587
	(line) energy: 39751.735318086925
(main) optimum:
       index: 2
       energy: 39751.46314724073
       force: 710.2794778485808
       upper_bound: 0.002716278033939681


(main) CYCLE START
	(line) energy: 39748.09768598653
	(line) energy: 39748.05350735164
	(line) energy: 39748.08710271654
	(line) energy: 39748.179441284185
	(line) energy: 39748.31065512402
(main) optimum:
       index: 2
       energy: 39748.05350735164
       force: 682.4390781570459
       upper_bound: 0.002576887603660062

(main) CYCLE START

	(line) energy: 39745.02309695805
	(line) energy: 39744.98555016918
	(line) energy: 39745.01459867834
	(line) energy: 39745.09594696746
	(line) energy: 39745.21245981006
(main) optimum:
       index: 2
       energy: 39744.98555016918
       force: 656.2342772033165
       upper_b

       upper_bound: 0.0009471075877519824

(main) CYCLE START

	(line) energy: 39712.02269810699
	(line) energy: 39712.05150756786
	(line) energy: 39712.08914980134
(main) optimum:
       index: 0
       energy: 39711.99312785291
       force: 326.7652100753863
       upper_bound: 0.0011092656172311322

(main) CYCLE START

	(line) energy: 39711.393369261794
	(line) energy: 39711.38223759683
	(line) energy: 39711.384547581205
	(line) energy: 39711.39750373655
	(line) energy: 39711.4184148634
(main) optimum:
       index: 2
       energy: 39711.38223759683
       force: 325.31120565976585
       upper_bound: 0.0010523417641688693

(main) CYCLE START

	(line) energy: 39710.805156932096
	(line) energy: 39710.79358449695
	(line) energy: 39710.80023245958
	(line) energy: 39710.810729097735
	(line) energy: 39710.83399903772
(main) optimum:
       energy: 39710.79358449695
       index: 2
       force: 308.6819137742745
       upper_bound: 0.000998339055508019

(main) CYCLE START

	(line) ener

In [43]:
# connect to descent output
%matplotlib
fig, ax = plt.subplots(2, 2, figsize=(10, 10))
def animate(i):
    global full_list
    try:
        obj = obj_q.get(False)
        
        if "energies" in obj:
            full_list.append(obj)
            ax[0, 0].clear()
            ax[0, 1].clear()
            ax[1, 0].clear()
            ax[0, 0].set_title("Energy (main)")
            ax[0, 1].set_title("RMSD (main)")
            ax[1, 0].set_title("Force (main)")
            n = len(obj["energies"])
            m = len(obj["forces"])
            k = len(obj["states"])
            if len(obj["energies"]) > 0:
                ax[0, 0].scatter(np.arange(0, n, 1), obj["energies"])
                state_0 = obj["states"][0]
                weights = np.ones((len(state_0),))
                ax[0, 1].scatter(np.arange(0, k, 1), [rmsd(state_0, s, weights) for s in obj["states"]])
                ax[1, 0].scatter(np.arange(0, m, 1), [np.linalg.norm(f) for f in obj["forces"]])
        else:
            ax[1, 1].clear()
            ax[1, 1].set_title("Energy (line)")
            n = len(obj["energies_line"])
            if n > 0:
                ax[1, 1].scatter(np.arange(0, n, 1), obj["energies_line"])
    except Empty:
        pass
anim = animation.FuncAnimation(fig, animate, 5000)

In [ ]:
uprocess.terminate()

In [121]:
# CA coparison
calpha_0 = drs._refine_prot_init.select("calpha").copy()
nmw2.set_position(np.zeros(10,))
calpha_1 = drs._refine_prot.select("calpha").copy()
nmw2.set_position(opt_pos)
calpha_2 = drs._refine_prot.select("calpha").copy()

nca = len(calpha_0)
print(nca)
weights1 = np.ones((nca,))

124


In [122]:
print("01", rmsd(calpha_0.getCoords(), calpha_1.getCoords(), weights1))
print("02", rmsd(calpha_0.getCoords(), calpha_2.getCoords(), weights1))
print("12", rmsd(calpha_1.getCoords(), calpha_2.getCoords(), weights1))

01 0.3893572220770151
02 0.1114435447861749
12 0.4136979486083795


Выходим из линейноо спуска как только энергия меньше начачльной
Нижнияя граница по рмсд = условие выхода из алгоритма.
Output пдб файл с моделями. Графики E(i) |F|(i), RMSD(A0, Ai).
Клешня.